Imports


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder 
from sklearn.preprocessing import OrdinalEncoder 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
%matplotlib inline

Arrumando Preço

In [2]:
def arruma(preco):
    if pd.isnull(preco):
        return CellDF['PREÇO'].median()  
    preco = preco.strip().replace('R$', '', 1)
    preco = preco.replace('.', '_')
    preco = preco.replace(',', '.')
    preco = preco.replace('_', '')
    return np.float32(preco)

sconverter = {
    'PREÇO': arruma,
}



CellDF = pd.read_csv('resultado_parcial.csv', converters=sconverter)

print(CellDF.info())


CellDF = CellDF.drop(CellDF[CellDF['PREÇO'] >= 9000].index)

CellDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 431 entries, 0 to 430
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   MARCA           429 non-null    object 
 1   MODELO          423 non-null    object 
 2   SISTEMA_OP      431 non-null    object 
 3   REDE            431 non-null    object 
 4   MEMORIA_INT     425 non-null    float64
 5   RAM             391 non-null    float64
 6   RESOL CAM TRAS  374 non-null    float64
 7   GARANTIA        399 non-null    float64
 8   PESO            429 non-null    float64
 9   PREÇO           431 non-null    float32
dtypes: float32(1), float64(5), object(4)
memory usage: 32.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 422 entries, 0 to 430
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   MARCA           421 non-null    object 
 1   MODELO          414 non-null    object 


Dropando os nulo, fds

In [3]:
dfamor = CellDF.dropna()

Tratando Modelos Zaralhados

In [4]:
dfamor['MARCA']=  dfamor['MARCA'].apply(lambda x: x.upper().strip()) 
dfamor['MARCA']=  dfamor['MARCA'].apply(lambda x: 'SAMSUNG' if x == 'GALAXY S22 PLUS' else x)
dfamor['MARCA']=  dfamor['MARCA'].apply(lambda x: 'SAMSUNG' if x.find('SAMSUNG') != -1 else x)
dfamor['MARCA']=  dfamor['MARCA'].apply(lambda x: 'MOTOROLA' if x.find('MOTOROLA') != -1 else x)

dfamor 

/tmp/ipykernel_47343/3817290528.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfamor['MARCA']=  dfamor['MARCA'].apply(lambda x: x.upper().strip())
/tmp/ipykernel_47343/3817290528.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfamor['MARCA']=  dfamor['MARCA'].apply(lambda x: 'SAMSUNG' if x == 'GALAXY S22 PLUS' else x)
/tmp/ipykernel_47343/3817290528.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

,MARCA,MODELO,SISTEMA_OP,REDE,MEMORIA_INT,RAM,RESOL CAM TRAS,GARANTIA,PESO,PREÇO
0,MOTOROLA,MOTO G 52,Android,4G,64.0,4.0,48.0,12.0,420.0,1089.000000
1,SAMSUNG,Galaxy A54,Android,5G,128.0,8.0,50.0,12.0,332.0,1859.000000
2,SAMSUNG,Galaxy A14,Android,5G,128.0,8.0,50.0,12.0,410.0,1129.000000
3,SAMSUNG,Galaxy A54,Android,5G,128.0,8.0,50.0,12.0,334.0,1859.000000
4,SAMSUNG,Galaxy A04e,Android,4G,64.0,4.0,13.0,12.0,284.0,759.000000
...,...,...,...,...,...,...,...,...,...,...
424,SAMSUNG,Galaxy S22+,Android,5G,128.0,8.0,50.0,12.0,327.0,6299.100098
425,SAMSUNG,Galaxy S22+,Android,5G,256.0,8.0,50.0,12.0,327.0,6749.100098
426,APPLE,iPhone 13,IOS,5G,256.0,6.0,12.0,12.0,1000.0,7547.000000
427,SAMSUNG,Galaxy S22+,Android,5G,128.0,8.0,50.0,12.0,413.0,4949.100098


In [5]:
dfamor['MARCA'].unique()

array(['MOTOROLA', 'SAMSUNG', 'APPLE', 'ASUS', 'POSITIVO', 'XIAOMI',
       'TCL', 'PHILCO', 'INFINIX', 'BLUE', 'MULTILASER', 'NOKIA', 'LG',
       'TECTOY', 'RED MOBILE', 'HUSKY SPORTS', 'REALME'], dtype=object)

Variável categórica -> numérica

In [6]:
print(dfamor['MARCA'].unique())
print(dfamor['SISTEMA_OP'].unique())
print(dfamor['REDE'].unique())

['MOTOROLA' 'SAMSUNG' 'APPLE' 'ASUS' 'POSITIVO' 'XIAOMI' 'TCL' 'PHILCO'
 'INFINIX' 'BLUE' 'MULTILASER' 'NOKIA' 'LG' 'TECTOY' 'RED MOBILE'
 'HUSKY SPORTS' 'REALME']
['Android' 'IOS']
['4G' '5G' '3G']


In [7]:
encoder = OneHotEncoder(handle_unknown='ignore', sparse=False, categories='auto', dtype='int8')
dataframeenc = pd.DataFrame(encoder.fit_transform(dfamor[['MARCA','SISTEMA_OP']]))
columns = encoder.get_feature_names_out(['MARCA','SISTEMA_OP'])
dataframeenc.columns = columns
dataframeenc.index = dfamor.index
dfamorepaixao = pd.concat([dfamor, dataframeenc], axis=1)
dfamorepaixao = dfamorepaixao.drop(['MARCA', 'SISTEMA_OP', 'MODELO'],axis=1)

dfamorepaixao

/home/renanxm/.local/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,REDE,MEMORIA_INT,RAM,RESOL CAM TRAS,GARANTIA,PESO,PREÇO,MARCA_APPLE,MARCA_ASUS,MARCA_BLUE,...,MARCA_PHILCO,MARCA_POSITIVO,MARCA_REALME,MARCA_RED MOBILE,MARCA_SAMSUNG,MARCA_TCL,MARCA_TECTOY,MARCA_XIAOMI,SISTEMA_OP_Android,SISTEMA_OP_IOS
0,4G,64.0,4.0,48.0,12.0,420.0,1089.000000,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,5G,128.0,8.0,50.0,12.0,332.0,1859.000000,0,0,0,...,0,0,0,0,1,0,0,0,1,0
2,5G,128.0,8.0,50.0,12.0,410.0,1129.000000,0,0,0,...,0,0,0,0,1,0,0,0,1,0
3,5G,128.0,8.0,50.0,12.0,334.0,1859.000000,0,0,0,...,0,0,0,0,1,0,0,0,1,0
4,4G,64.0,4.0,13.0,12.0,284.0,759.000000,0,0,0,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,5G,128.0,8.0,50.0,12.0,327.0,6299.100098,0,0,0,...,0,0,0,0,1,0,0,0,1,0
425,5G,256.0,8.0,50.0,12.0,327.0,6749.100098,0,0,0,...,0,0,0,0,1,0,0,0,1,0
426,5G,256.0,6.0,12.0,12.0,1000.0,7547.000000,1,0,0,...,0,0,0,0,0,0,0,0,0,1
427,5G,128.0,8.0,50.0,12.0,413.0,4949.100098,0,0,0,...,0,0,0,0,1,0,0,0,1,0


In [8]:
maping = {
    '3G': 0,
    '4G': 1,
    '5G': 2
}

dfamorepaixao['REDE'] = dfamorepaixao['REDE'].replace(maping)

dfamorepaixao['REDE'].unique()

array([1, 2, 0])

In [9]:
dfamorepaixao.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 309 entries, 0 to 428
Data columns (total 26 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   REDE                309 non-null    int64  
 1   MEMORIA_INT         309 non-null    float64
 2   RAM                 309 non-null    float64
 3   RESOL CAM TRAS      309 non-null    float64
 4   GARANTIA            309 non-null    float64
 5   PESO                309 non-null    float64
 6   PREÇO               309 non-null    float32
 7   MARCA_APPLE         309 non-null    int8   
 8   MARCA_ASUS          309 non-null    int8   
 9   MARCA_BLUE          309 non-null    int8   
 10  MARCA_HUSKY SPORTS  309 non-null    int8   
 11  MARCA_INFINIX       309 non-null    int8   
 12  MARCA_LG            309 non-null    int8   
 13  MARCA_MOTOROLA      309 non-null    int8   
 14  MARCA_MULTILASER    309 non-null    int8   
 15  MARCA_NOKIA         309 non-null    int8   
 16  MARCA_PH

Treino e Teste + Normalização

In [10]:
dfamorepaixao.columns

x = (dfamorepaixao.drop('PREÇO', axis = 1)).to_numpy()
y = (dfamorepaixao['PREÇO']).to_numpy()

xtreino, xteste, ytreino, yteste = train_test_split(x,y, train_size=0.7)

In [11]:
Scaler = MinMaxScaler()
columns = xtreino
display(yteste)
xtreino = Scaler.fit_transform(xtreino)
xteste = Scaler.transform(xteste)


Scaleroutro = MinMaxScaler()
ytreino = Scaleroutro.fit_transform(ytreino.reshape(-1,1))
yteste = Scaleroutro.transform(yteste.reshape(-1,1))



display(ytreino)

array([ 385.9 ,  989.9 , 4859.1 , 4299.  , 3069.4 , 1944.99, 2998.  ,
       6209.1 ,  879.  ,  899.99, 1543.69,  989.  , 5199.  , 3597.37,
       1799.1 , 1999.99, 1399.99, 1110.59,  982.79, 1202.94, 2204.1 ,
       6209.1 , 2531.  , 6999.  , 1129.  , 1259.99, 1337.97,  929.69,
       8567.  ,  539.99, 2659.05, 1998.76, 1605.  ,  854.99,  989.1 ,
       7199.1 , 2599.2 , 6999.  , 1289.  , 2998.  , 3599.99, 5309.1 ,
       3361.49, 1215.4 , 1079.1 , 2519.1 , 1290.9 , 7210.  , 1166.6 ,
       1299.  , 6150.  , 3235.26, 3199.  , 1290.9 ,  375.21, 1399.  ,
       6209.1 , 2662.19, 3569.  , 7839.  , 1749.  , 1169.99, 3799.  ,
       1131.97, 1166.6 , 1439.1 , 2339.99, 3597.37,  596.69,  899.1 ,
       6999.  ,  759.  ,  899.99, 2743.  , 4599.  , 3199.  , 1859.  ,
        645.05, 3799.99, 7598.1 , 2899.99,  906.99, 2999.99, 2599.99,
       3397.97, 1091.55, 1199.  ,  768.  , 3799.  ,  806.53,  629.99,
       4299.  ,  685.8 ], dtype=float32)

array([[0.2072722 ],
       [0.07616913],
       [0.25542918],
       [0.19348386],
       [0.53464323],
       [0.21682179],
       [0.19578192],
       [0.91955644],
       [0.5173963 ],
       [0.15471102],
       [0.22894746],
       [0.09947145],
       [0.99999994],
       [0.09007123],
       [0.07385959],
       [0.04639779],
       [0.07972997],
       [0.5633575 ],
       [0.22440538],
       [0.45718604],
       [0.06400092],
       [0.22440538],
       [0.13746639],
       [0.09205447],
       [0.6104677 ],
       [0.65527976],
       [0.4944157 ],
       [0.03485234],
       [0.45994484],
       [0.11592439],
       [0.04640813],
       [0.98551077],
       [0.08272894],
       [0.16039182],
       [0.10028726],
       [0.05962312],
       [0.21754336],
       [0.1382167 ],
       [0.6897622 ],
       [0.7042284 ],
       [0.839021  ],
       [0.6794209 ],
       [0.1893347 ],
       [0.06938872],
       [0.09007123],
       [0.26185223],
       [0.3071355 ],
       [0.032

In [12]:
model = LinearRegression()

model.fit(xtreino, ytreino)

prediction = model.predict(xteste)

yteste = Scaleroutro.inverse_transform(yteste)

prediction = Scaleroutro.inverse_transform(prediction)

mean_squared_error(yteste, prediction)


print(np.sqrt(mean_squared_error(yteste, prediction)))

1303.2189939945813
